## Environment Setup

In [1]:
import os
from dotenv import load_dotenv 
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_cohere import CohereEmbeddings
from langchain_cohere import ChatCohere
from langchain.chains import RetrievalQA
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone


c:\Users\Anthony\Documents\pinecone_demo_github\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Anthony\Documents\pinecone_demo_github\venv\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStor

In [2]:
load_dotenv()

True

## load document 

In [3]:
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents

In [4]:
docs=read_doc('documents/')


In [5]:
def chunk_data(docs,chunk_size=100,chunk_overlap=20):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return doc

## split text into chunks

In [6]:
documents = chunk_data(docs=docs)

## embeddings and vector store

In [7]:
embeddings=CohereEmbeddings(model="embed-english-light-v3.0",
                            cohere_api_key=os.environ['COHERE_API_KEY'])
embeddings

CohereEmbeddings(client=<cohere.client.Client object at 0x000001FE248CEA50>, async_client=<cohere.client.AsyncClient object at 0x000001FE24F1EBA0>, model='embed-english-light-v3.0', truncate=None, cohere_api_key=SecretStr('**********'), embedding_types=['float'], max_retries=3, request_timeout=None, user_agent='langchain:partner', base_url=None)

In [8]:
vectors=embeddings.embed_query("what does the writer love about the recipient?")

In [9]:
len(vectors)

384

In [10]:


pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'])

index_name = "pineconedemo"

if not pc.has_index(index_name):
    pc.create_index_for_model(
        name=index_name,
        cloud="aws",
        region="us-east-1",
        embed={
            "model":"llama-text-embed-v2",
            "field_map":{"text": "chunk_text"}
        }
    )

## query the vector database

In [11]:
vectorstore_from_docs = PineconeVectorStore.from_documents(
        documents,
        index_name=index_name,
        embedding=embeddings
    )
retriever = vectorstore_from_docs.as_retriever()

In [12]:


llm = ChatCohere(model="command-r", cohere_api_key=os.environ['COHERE_API_KEY'])


In [13]:
system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise.\n\n"
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [14]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)


In [15]:
query = "what does the writer love about the recipient?"
response = chain.invoke({"input": query})
#print(response)